In [1]:
import numpy as np
import itertools as itr

import os as os
import sys as sys 
import pandas as pd
import warnings
import time as time
import random
from __future__ import print_function # so print doesn't show brackets

sys.path.append(os.path.join("..","Libraries","QML_lib"))
import Evo as evo
import DataBase 
import QMD
import QML
import ModelGeneration
import BayesF
import matplotlib.pyplot as plt

global paulis_list
paulis_list = {'i' : np.eye(2), 'x' : evo.sigmax(), 'y' : evo.sigmay(), 'z' : evo.sigmaz()}

warnings.filterwarnings("ignore", message='Negative weights occured', category=RuntimeWarning)



class NewQMDClass():
    #TODO: rename as ModelsDevelopmentClass when finished
    def __init__(self,
                 initial_op_list,
                 true_operator='x',
                 maxmodnum=3, 
                 max_dimension=4, 
                 gaussian=True,
                 max_num_layers = 10,
                 max_num_branches = 20 
                ):
#    def __init__(self, initial_op_list, true_op_list, true_param_list):
        
        trueOp = DataBase.operator(true_operator)
        self.TrueOpName = true_operator
        self.MaxDimension = max_dimension
        self.TrueOpDim = trueOp.num_qubits
        self.InitialOpList = initial_op_list
        self.TrueOpList = trueOp.constituents_operators
#        self.TrueParamsList = [random.random() for i in self.TrueOpList] # TODO: actual true params?
        # todo set true parmams properly
        self.TrueParamsList = [0.75 for i in self.TrueOpList] # TODO: actual true params?
        #self.TrueHam = evo.getH(self.TrueParamsList, self.TrueOpList)
        self.TrueHam = np.tensordot(self.TrueParamsList, self.TrueOpList, axes=1)
        self.MaxModNum = maxmodnum #TODO: necessary?
        self.gaussian = gaussian
        self.NumModels = len(initial_op_list)
        self.NumParticles = 1000
        self.MaxBranchID = max_num_branches
        self.HighestBranchID = 0
        self.MaxLayerNumber = max_num_layers
        self.BranchChampions = {}
        self.LayerChampions = {}
        self.BayesFactorsByBranch ={}
        self.BranchRankings = {}
        self.InterBranchChampions = {}
        self.GlobalEpoch = 0 
        
#        for i in range(self.MaxBranchID+1):
#            self.BranchChampions[i] = 0
#        for i in range(self.MaxLayerNumber+1):
#            self.LayerChampions[i] = 0
        
        print("\nQMD has true operator: ", true_operator)
        print("QMD has true params : ", self.TrueParamsList, "\n")

        # Initialise database and lists.
        self.initiateDB()
        
    def initiateDB(self):
        
        ## TODO: Models should be initialised with appropriate TrueOp dimension -- see getListTrueOpByDimension function
        self.db, self.legacy_db, self.model_lists = \
            DataBase.launch_db(
                true_op_name = self.TrueOpName,
                gen_list = self.InitialOpList,
                true_ops = self.TrueOpList,
                true_params = self.TrueParamsList
            )
    def addModel(self, model, treeID=0):
        self.NumModels += int(1)
        tryAddModel = DataBase.add_model(
            model_name = model,
            running_database = self.db,
            num_particles = self.NumParticles, 
            true_op_name = self.TrueOpName,
            model_lists = self.model_lists,
            true_ops = self.TrueOpList,
            true_params = self.TrueParamsList,
            treeID = treeID,
            modelID = int(self.NumModels),
        )
        
        if tryAddModel is False: 
            self.NumModels-=int(1)

    def newBranch(self, model_list):
        self.HighestBranchID += 1
        branchID = self.HighestBranchID
        for model in model_list:
            self.addModel(model, treeID=branchID)
    
    def printState(self):
        print("Branch champions: \n", self.BranchChampions)
        print("InterBranch champions: \n", self.InterBranchChampions)
        print("Branch Rankings: \n", self.BranchRankings)
        #print("Layer Champions: \n", self.LayerChampions)
            
            
    def getModelInstance(self, name):
        try: 
            instance = DataBase.get_qml_instance(self.db, name)
            return instance
        except: 
            if name in list(self.legacy_db['<Name>']):
                print("Operator in legacy databse - retired. ")
            else: 
                print("Model not found.")
    def getOperatorInstance(self, name):
        try: 
            return DataBase.get_operator_instance(self.db, name)
        except:
            if name in list(self.legacy_db['<Name>']):
                print("Operator in legacy databse - retired. ")
            else: 
                print("Operator not found.")

    def getModelDBIndex(self, name):
        return DataBase.get_location(self.db, name)

    def getModelInstanceFromID(self, model_id):
        return DataBase.model_instance_from_id(self.db, model_id)    
    
    def killModel(self, name):
        if name not in list(self.db['<Name>']):
            print("Cannot remove ", name, "; not in ", list(self.db["<Name>"]))
        else:
            print("Killing model", name)
            # Add to legacy_db
            DataBase.move_to_legacy(self.db, self.legacy_db, name)
            model_instance = self.getModelInstance(name)
            operator_instance = self.getOperatorInstance(name)
            # Remove from self.db
            self.db = DataBase.remove_model(self.db, name)
            del model_instance
            del operator_instance
    
            #TODO: plot?

    def runIQLE(self, model, num_exp=50):
        model_exists=False
        if model in list(self.db['<Name>']):
            model_exists = True
        elif model in list(self.legacy_db['<Name>']):
            print("Model ", model, " previously considered and retired.")
            
        if model_exists: 
            model_instance = self.getModelInstance(model)
            model_instance.UpdateModel(num_exp)
            self.updateModelRecord(name=model, field='Completed', new_value=True)
            #model_instance.BayesOnModelsWithinTrees
        else: 
            print("Model ", model ,"does not exist")

    def runAllActiveModelsIQLE(self, num_exp):
        active_models = self.db.loc[self.db['Status']=='Active']['<Name>']

        for model in active_models:
            self.runIQLE(model=model, num_exp=num_exp)
        self.GlobalEpoch += num_exp
            
        
    def updateModelRecord(self, field, name=None, model_id=None, new_value=None, increment=None):
        DataBase.update_field(
            db=self.db, 
            name=name,
            model_id=model_id,
            field=field,
            new_value=new_value,
            increment=increment
        )
    def pullField(self, name, field):
        return DataBase.pull_field(self.db, name, field)

    def statusChangeBranch(self, branchID, new_status='Saturated'):
        self.db.loc[ self.db['TreeID']==branchID , 'Status'] = new_status

    def statusChangeModel(self, model_name, new_status='Saturated'):
        self.db.loc[ self.db['<Name>']==model_name , 'Status'] = new_status
        
    def getListTrueOpByDimension(self):
        self.TrueOpListByDim = {}
        self.TrueParamByDim = {}
        for dim in range(1, 1+self.MaxDimension):
            new_op = ModelGeneration.identity_interact(subsystem=self.TrueOpName, num_qubits=dim, return_operator=True)
            self.TrueOpListByDim[dim] = new_op.constituents_operators
        for i in range(1, self.TrueOpDim+1):
            self.TrueParamByDim[i] = self.TrueParamsList
        for i in range(self.TrueOpDim+1, self.MaxDimension):
            self.TrueParamByDim[i] = [self.TrueParamsList[0]]

    def compareModels(self, log_comparison_high=50.0, num_times_to_use = 'all', model_a_id = None, model_b_id =None, model_a = None, model_b = None, name_a=None, name_b=None):
        # Either pass in name_a and name_b OR model_a and model_b
        if model_a is None and model_b is None:
            if model_a_id is not None and model_b_id is not None: 
                model_a = self.getModelInstanceFromID(model_a_id)
                model_b = self.getModelInstanceFromID(model_b_id)
            else: # if only names were passed 
                model_a = self.getModelInstance(name_a)
                model_b = self.getModelInstance(name_b)
        if model_a ==  model_b:
            print("Same models, don't need to compare.")
            return "Same Models"

        log_comparison_low = 1/log_comparison_high
        
        if num_times_to_use == 'all':
            times_a = model_a.TrackTime
        elif len(model_a.TrackTime) < num_times_to_use:
            times_a = model_a.TrackTime
        else: 
            times_a = model_a.TrackTime[num_times_to_use:]
        
        if num_times_to_use=='all':
            times_b = model_b.TrackTime
        elif len(model_b.TrackTime) < num_times_to_use:
            times_b = model_b.TrackTime
        else: 
            times_b = model_b.TrackTime[num_times_to_use:]
        gen_a = model_a.GenSimModel
        gen_b = model_b.GenSimModel

        times = times_a + times_b
        exps_a = get_exps(model_a, gen_a, times)
        exps_b = get_exps(model_b, gen_b, times)

        log_l_a =  get_log_likelihood(model_a, gen_a, exps_a)
        log_l_b =  get_log_likelihood(model_b, gen_b, exps_b)

        bayes_factor = np.exp(log_l_a - log_l_b)

        print("bayes_factor of log likelihoods between ", model_a.Name, " and ", model_b.Name, " = ", bayes_factor)
        if bayes_factor > log_comparison_high: 
            print("Point to ", model_a.Name)
            return "a"
        elif bayes_factor < log_comparison_low: 
            print("Point to ", model_b.Name)
            return "b"
        else:
            print("No real winner")  
        # todo: Add bayes_factor with mod_id's to QML class

            
    def compareModelsWithinBranch(self, treeID):
        active_models_in_tree = DataBase.active_model_ids_by_tree_id(self.db, treeID)
        
        models_points = {}
        for model_id in active_models_in_tree:
            models_points[model_id] = 0
        
        for i in range(len(active_models_in_tree)):
            mod1 = active_models_in_tree[i]
            for j in range(i, len(active_models_in_tree)): 
                mod2 = active_models_in_tree[j]
                res = self.compareModels(model_a_id = mod1, model_b_id=mod2)
                if res == "a":
                    models_points[mod1] += 1
                elif res == "b":
                    models_points[mod2] += 1
                    # todo if more than one model has max points
                    
        max_points = max(models_points.values())
        max_points_branches = [key for key, val in models_points.items() if val==max_points]
        if len(max_points_branches) > 1: 
            # todo: recompare. Fnc: compareListOfModels (rather than branch based)
            champ_id = self.compareModelList(max_points_branches, bayes_threshold=1, models_points_dict=models_points)
        else: 
            champ_id = max(models_points, key=models_points.get)
        champ_name = DataBase.model_name_from_id(self.db, champ_id)
        
        #todo list of ranked models by branch
        
        self.BranchChampions[int(treeID)] = champ_id
        for model_id in active_models_in_tree:
            self.updateModelRecord(model_id=model_id, field='Status', new_value='Deactivated')
        self.updateModelRecord(name=DataBase.model_name_from_id(self.db, champ_id), field='Status', new_value='Active')

# sorted_keys = sorted(my_dict, key=my_dict.get, reverse=True)
        ranked_model_list = sorted_keys = sorted(models_points, key=models_points.get, reverse=True)
        print("Branch: ", treeID)
        print("List : ", ranked_model_list)
        print("Models_points dict : ", models_points)
        self.BranchRankings[treeID] = ranked_model_list
        
        print("Champion of tree ", treeID, " is ", champ_name)
        return models_points, champ_id

    
    def compareModelList(self, model_list, bayes_threshold = 50, models_points_dict=None):
        models_points = {}
        for mod in model_list:
            models_points[mod] = 0
        
        for i in range(len(model_list)):
            mod1 = model_list[i]
            for j in range(i, len(model_list)):
                mod2 = model_list[j]
                if mod1 != mod2:
                    res = self.compareModels(model_a_id=mod1, model_b_id=mod2, log_comparison_high=bayes_threshold)
                    if res == "a":
                        models_points[mod1] += 1
                        if models_points_dict is not None: 
                            models_points_dict[mod1]+=1
                    elif res == "b":
                        models_points[mod2]+=1
                        if models_points_dict is not None: 
                            models_points_dict[mod2]+=1

        max_points = max(models_points.values())
        max_points_branches = [key for key, val in models_points.items() if val==max_points]
        if len(max_points_branches) > 1: 
            # todo: recompare. Fnc: compareListOfModels (rather than branch based)
            print("No distinct champion, recomputing bayes factors between : ", max_points_branches)
            champ_id = self.compareModelList(max_points_branches, bayes_threshold=1)
        else: 
            champ_id = max(models_points, key=models_points.get)
        champ_name = DataBase.model_name_from_id(self.db, champ_id)

        
        return champ_id
    
    def interBranchChampion(self, branch_list=[], global_champion=False):
        all_branches = self.db['TreeID'].unique()
        if global_champion == True: 
            trees = all_branches
        else: 
            trees = branch_list
        
        num_branches = len(trees)
        points_by_branches = [None] * num_branches
        champions_of_branches = [None] * num_branches

        for i in range(num_branches):
            treeID = trees[i]
            if treeID not in all_branches:
                print("Tree ID : ", treeID)
                warnings.warn("Tree not in database.")
                return False
            points_by_branches[i], champions_of_branches[i] = qmd.compareModelsWithinBranch(treeID)

        branch_champions_points = {}
        for c in champions_of_branches: 
            branch_champions_points[c] = 0

        for i in range(num_branches):
            mod1 = champions_of_branches[i]
            for j in range(i, num_branches):
                mod2 = champions_of_branches[j]
                if mod1!=mod2:
                    res = self.compareModels(model_a_id=mod1, model_b_id=mod2, log_comparison_high=20.0)
                    if res == "a":
                        branch_champions_points[mod1] += 1
                    elif res == "b":
                        branch_champions_points[mod2] += 1
        self.ranked_champions = sorted(branch_champions_points, reverse=True)
        
        max_points = max(branch_champions_points.values())
        max_points_branches = [key for key, val in branch_champions_points.items() if val==max_points]
        if len(max_points_branches) > 1: 
            # todo: recompare. Fnc: compareListOfModels (rather than branch based)
            print("No distinct champion, recomputing bayes factors between : ", max_points_branches)
            champ_id = self.compareModelList(max_points_branches, bayes_threshold=1, models_points_dict=branch_champions_points)
        else: 
            champ_id = max(branch_champions_points, key=branch_champions_points.get)
        champ_name = DataBase.model_name_from_id(self.db, champ_id)
        
        branch_champ_names = [DataBase.model_name_from_id(self.db, mod_id) for mod_id in champions_of_branches]
        self.statusChangeModel(champ_name, new_status = 'Active')
        
        interBranchChampListID = len(self.InterBranchChampions)
        self.InterBranchChampions[interBranchChampListID] = [trees, champ_id]
        print("Champion of branches ", trees, " is ",  champ_name)
        return champ_name, branch_champ_names
    
    def globalChampionCalculation(self):
        trees = self.db['TreeID'].unique()
        
        num_trees = len(trees)
        self.points_by_trees = [None] * num_trees
        self.champions_of_trees = [None] * num_trees

        for i in range(num_trees):
            treeID = trees[i]
            self.points_by_trees[i], self.champions_of_trees[i] = qmd.compareModelsWithinBranch(treeID)

        self.champions_points = {}
        for c in self.champions_of_trees: 
            self.champions_points[c] = 0

        for i in range(num_trees):
            mod1 = self.champions_of_trees[i]
            for j in range(i, num_trees):
                mod2 = self.champions_of_trees[j]
                if mod1!=mod2:
                    res = self.compareModels(model_a_id=mod1, model_b_id=mod2, log_comparison_high=10.0)
                    if res == "a":
                        self.champions_points[mod1] += 1
                    elif res == "b":
                        self.champions_points[mod2]+=1
        self.ranked_champions = sorted(self.champions_points, reverse=True)
        champ_id = max(self.champions_points, key=self.champions_points.get)
        champ_name = DataBase.model_name_from_id(self.db, champ_id)
        print("Champion of Champions is",  champ_name)
        
        
    def spawn(self, 
              branch_list = None, 
              num_models_to_consider=1, 
              absolute_champion=False, 
              all_branches=False
             ):

        if all_branches or branch_list is None: 
            global_champion = True
            
        overall_champ, branch_champions = self.interBranchChampion(branch_list=branch_list, global_champion=global_champion)

        print("Overall champ : ", overall_champ)
        if absolute_champion:
            print("Only using absolute champion")
            new_models = ModelGeneration.new_model_list(model_list=[overall_champ], generator='simple_ising')
        else: 
            print("Using branch champions list")
            print("branch champions : ", branch_champions)
            new_models = ModelGeneration.new_model_list(model_list=branch_champions, generator='simple_ising')
        
        print("New models to add to new branch : ", new_models)
        qmd.newBranch(model_list=new_models) 
        
        #todo probailistically append model_list with suboptimal model in any of the branches in branch_list

        
def get_exps(model, gen, times):

    exps = np.empty(len(times), dtype=gen.expparams_dtype)
    exps['t'] = times

    for i in range(1, len(gen.expparams_dtype)):
        col_name = 'w_'+str(i)
        exps[col_name] = model.FinalParams[i-1,0] ## TODO: should be model.NewEval[i-1]???
    return exps

def get_log_likelihood(model, gen, exps):
    import copy
    updater = copy.deepcopy(model.Updater)
    data = gen.simulate_experiment(model.SimParams, exps)[0][0]
    updater.batch_update(data, exps, resample_interval=100)
    log_likelihood = np.sum(updater.log_total_likelihood)
    del updater
    return log_likelihood
            

    

/usr/local/lib/python2.7/dist-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)
/usr/local/lib/python2.7/dist-packages/qinfer/parallel.py:52: UserWarning: Could not import IPython parallel. Parallelization support will be disabled.
  "Could not import IPython parallel. "


In [2]:
import random
num_exp = 10

max_dim=2
dims = range(1,max_dim+1)
num_qubits=random.choice(dims)

true_op = 'xTx'
generators = ['x', 'y']
# above doesn't 

branch_1 = ['xPy', 'z', 'zMy']
#true_op = ModelGeneration.random_ising_chain(num_dimensions=num_qubits, num_directions = 1, include_identity=False)
#true_op = random.choice(generators)
print("True operator : ", true_op)

qmd= NewQMDClass(initial_op_list=generators, true_operator=true_op)
#qmd.newBranch(branch_1)

qmd.runAllActiveModelsIQLE(num_exp=num_exp)

qmd.spawn()

#qmd.spawn(branch_list = [0,1], absolute_champion=False)
qmd.runAllActiveModelsIQLE(num_exp=num_exp)
#qmd.spawn(branch_list = [2], absolute_champion=False)




True operator :  xTx

QMD has true operator:  xTx
QMD has true params :  [0.75] 

Before dimensionalising name. Name =  x true_dim =  2
Model  x  not previously considered -- adding.
Initialization Ready
Before dimensionalising name. Name =  y true_dim =  2
Model  y  not previously considered -- adding.
Initialization Ready
Initial time selected > 1.43711062617
Probe state =  [-0.42171720-0.30248722j -0.10693239+0.34907196j  0.61726598-0.12373849j
 -0.23887740-0.37944618j]
Probe state =  [-0.42171720-0.30248722j -0.10693239+0.34907196j  0.61726598-0.12373849j
 -0.23887740-0.37944618j]
Probe state =  [-0.42171720-0.30248722j -0.10693239+0.34907196j  0.61726598-0.12373849j
 -0.23887740-0.37944618j]
Probe state =  [-0.42171720-0.30248722j -0.10693239+0.34907196j  0.61726598-0.12373849j
 -0.23887740-0.37944618j]
Probe state =  [-0.42171720-0.30248722j -0.10693239+0.34907196j  0.61726598-0.12373849j
 -0.23887740-0.37944618j]
Probe state =  [-0.42171720-0.30248722j -0.10693239+0.34907196j  0

Probe state =  [-0.60495390-0.28600867j  0.40176608-0.18291544j -0.31243992+0.03136327j
  0.37181405+0.34714224j]
Probe state =  [-0.60495390-0.28600867j  0.40176608-0.18291544j -0.31243992+0.03136327j
  0.37181405+0.34714224j]
Probe state =  [-0.60495390-0.28600867j  0.40176608-0.18291544j -0.31243992+0.03136327j
  0.37181405+0.34714224j]
Probe state =  [-0.60495390-0.28600867j  0.40176608-0.18291544j -0.31243992+0.03136327j
  0.37181405+0.34714224j]
Probe state =  [-0.60495390-0.28600867j  0.40176608-0.18291544j -0.31243992+0.03136327j
  0.37181405+0.34714224j]
Probe state =  [-0.60495390-0.28600867j  0.40176608-0.18291544j -0.31243992+0.03136327j
  0.37181405+0.34714224j]
Probe state =  [-0.60495390-0.28600867j  0.40176608-0.18291544j -0.31243992+0.03136327j
  0.37181405+0.34714224j]
Probe state =  [-0.60495390-0.28600867j  0.40176608-0.18291544j -0.31243992+0.03136327j
  0.37181405+0.34714224j]
Probe state =  [-0.60495390-0.28600867j  0.40176608-0.18291544j -0.31243992+0.03136327j


Probe state =  [-0.26353258+0.49213506j  0.04440220-0.48373586j  0.02212583-0.47380228j
  0.32993554-0.34430527j]
Probe state =  [-0.26353258+0.49213506j  0.04440220-0.48373586j  0.02212583-0.47380228j
  0.32993554-0.34430527j]
Probe state =  [-0.26353258+0.49213506j  0.04440220-0.48373586j  0.02212583-0.47380228j
  0.32993554-0.34430527j]
Probe state =  [-0.26353258+0.49213506j  0.04440220-0.48373586j  0.02212583-0.47380228j
  0.32993554-0.34430527j]
Probe state =  [-0.26353258+0.49213506j  0.04440220-0.48373586j  0.02212583-0.47380228j
  0.32993554-0.34430527j]
Probe state =  [-0.26353258+0.49213506j  0.04440220-0.48373586j  0.02212583-0.47380228j
  0.32993554-0.34430527j]
Final time selected > 0.429958600694
Final Parameters mean and stdev:[ 1.0341548   0.22889441]


In [4]:
qmd.compareModels(model_a_id=0, model_b_id=1)

bayes_factor of log likelihoods between  xTi  and  yTi  =  0.00021082871386
Point to  yTi


'b'

In [3]:
qmd.printState()

Branch champions: 
 {0: 1.0}
InterBranch champions: 
 {0: [array([ 0.]), 1.0]}
Branch Rankings: 
 {0.0: [1.0, 0.0]}


In [5]:
qmd.db

,<Name>,Completed,Epoch_Start,ModelID,Model_Class_Instance,Operator_Instance,Status,TreeID
0,x,True,0.0,0.0,<QML.ModelLearningClass instance at 0x7fefb03a...,<DataBase.operator instance at 0x7fefb03aec68>,Deactivated,0.0
1,y,True,0.0,1.0,<QML.ModelLearningClass instance at 0x7fef6e5c...,<DataBase.operator instance at 0x7fef6e5c7c20>,Active,0.0
2,yTx,True,0.0,3.0,<QML.ModelLearningClass instance at 0x7fefb032...,<DataBase.operator instance at 0x7fef6e5dde18>,Active,1.0
3,yTy,True,0.0,4.0,<QML.ModelLearningClass instance at 0x7fefb03a...,<DataBase.operator instance at 0x7fefb03ae098>,Active,1.0
4,yTz,True,0.0,5.0,<QML.ModelLearningClass instance at 0x7fefb032...,<DataBase.operator instance at 0x7fefb0322f38>,Active,1.0


In [ ]:
import random
num_exp = 50

max_dim=2
dims = range(1,max_dim+1)
num_qubits=random.choice(dims)

#true_op = 'xTx'
#generators = ['x', 'y', 'z', 'xTx']
# above doesn't 

true_op = 'xTxTTx'
generators = ['xTx', 'yTy', 'zTz']

branch_1 = ['xPy', 'z', 'zMy']
#true_op = ModelGeneration.random_ising_chain(num_dimensions=num_qubits, num_directions = 1, include_identity=False)
#true_op = random.choice(generators)
print("True operator : ", true_op)

qmd= NewQMDClass(initial_op_list=generators, true_operator=true_op)
#qmd.newBranch(branch_1)

qmd.runAllActiveModelsIQLE(num_exp=num_exp)

qmd.spawn()

#qmd.spawn(branch_list = [0,1], absolute_champion=False)
#qmd.runAllActiveModelsIQLE(num_exp=num_exp)
#qmd.spawn(branch_list = [2], absolute_champion=False)


In [14]:
def show_matrix(name):
    op = DataBase.operator(name)
    print(op.matrix)

In [12]:
name = 'yTi'
op = DataBase.operator(name)
print(op.matrix)

[[ 0.+0.j  0.+0.j  0.-1.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.-1.j]
 [ 0.+1.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+1.j  0.+0.j  0.+0.j]]


In [13]:
name = 'xTi'
op = DataBase.operator(name)
print(op.matrix)

[[ 0.+0.j  0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.+0.j]
 [ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]]


In [15]:
show_matrix('xTx')

[[ 0.+0.j  0.+0.j  0.+0.j  1.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]]


In [4]:
qmd.printState()

Branch champions: 
 {0: 1.0}
InterBranch champions: 
 {0: [array([ 0.]), 1.0]}
Branch Rankings: 
 {0.0: [1.0, 2.0, 0.0]}


In [8]:
qmd.printState()

Branch champions: 
 {0: 3.0}
InterBranch champions: 
 {0: [array([ 0.]), 3.0]}
Branch Rankings: 
 {0.0: [3.0, 0.0, 1.0, 5.0, 4.0, 6.0, 2.0]}


In [16]:
qmd.db

,<Name>,Completed,Epoch_Start,ModelID,Model_Class_Instance,Operator_Instance,Status,TreeID
0,x,True,0.0,0.0,<QML.ModelLearningClass instance at 0x7f1f62d9...,<DataBase.operator instance at 0x7f1f62d9b4d0>,Deactivated,0.0
1,y,True,0.0,1.0,<QML.ModelLearningClass instance at 0x7f1f8d26...,<DataBase.operator instance at 0x7f1f8d262050>,Deactivated,0.0
2,z,True,0.0,2.0,<QML.ModelLearningClass instance at 0x7f1f746e...,<DataBase.operator instance at 0x7f1f6a37acb0>,Active,0.0
3,zTx,True,0.0,4.0,<QML.ModelLearningClass instance at 0x7f1f62d9...,<DataBase.operator instance at 0x7f1fa424a2d8>,Active,1.0
4,zTy,True,0.0,5.0,<QML.ModelLearningClass instance at 0x7f1fa435...,<DataBase.operator instance at 0x7f1f8d29bbd8>,Active,1.0
5,zTz,True,0.0,6.0,<QML.ModelLearningClass instance at 0x7f1f62da...,<DataBase.operator instance at 0x7f1f6b186ea8>,Active,1.0


In [3]:
x=qmd.getModelInstance('x')
x.ProbeList

[array([ 0.31213127+0.42864342j, -0.22244595+0.57343059j,
        -0.40264863+0.30890865j,  0.28178671-0.05983165j]),
 array([ 0.43521611-0.09204417j, -0.35039304-0.5728932j ,
        -0.36607882+0.41092392j,  0.12216544-0.18258245j]),
 array([ 0.02142573+0.37240181j, -0.01878211+0.40322866j,
        -0.65081352+0.35588821j,  0.36260489-0.12733716j]),
 array([ 0.27507154+0.30136962j, -0.49866127-0.3580782j ,
         0.57003705+0.0678175j ,  0.32340977+0.1499787j ]),
 array([ 0.39018482+0.31400712j,  0.02238324+0.52499807j,
        -0.51439911+0.23981568j, -0.08611061-0.37881172j]),
 array([-0.21673713+0.59033959j,  0.46563151+0.25660968j,
         0.49059186-0.21911334j,  0.09114472+0.15768505j]),
 array([ 0.27367544-0.54216726j, -0.10601450-0.43203785j,
        -0.11964889-0.34488531j, -0.40732207+0.36617979j]),
 array([-0.43583493-0.50590487j,  0.45213460+0.35903037j,
         0.15792206-0.1363945j ,  0.29950045+0.29586535j]),
 array([-0.23184620-0.21007626j, -0.42097231+0.40448058j

In [5]:
x.TrackEval

[array([ 0.49589849]), array([ 0.10693542]), array([-0.00036283])]

## Brach generation development

In [ ]:
generate_new_model_list(generator_list=['xTy', 'yTx', 'y', 'xTyTTy'])

In [ ]:
def ising_fully_interacting(num_qubits):
    # fully interacting Ising Hamiltonian name
    this_idx = 0
    max_idx = 0
    for i in range(num_qubits):
        max_idx += i ## Triangle number of 
    p_str = ''
    for a in range(num_qubits):
        p_str += 'P'
    
    running_str=''
    for i in range(num_qubits):
        for j in range(i, num_qubits):
            if i!=j:
                this_idx+=1
                op_list = ['z', 'z']
                qub_list = [i,j]
                new_term = ModelGeneration.interaction_ham(qubit_list=qub_list, operator_list=op_list, num_qubits=num_qubits)
                running_str += new_term
                if(this_idx < max_idx):
                    running_str += p_str
    return running_str


In [ ]:
ModelGeneration.random_ising_name(num_dimensions=2, num_terms=2, num_directions=2, include_identity=True)

In [ ]:
ModelGeneration.

## Model comparison development

In [ ]:
treeTracker ={}
treeIDs = list(qmd.db['TreeID'].unique())

for tree in treeIDs:
    treeTracker[tree] = {}
    treeTracker[tree]['models'] = {}
    treeTracker[tree]['ModelIDs'] = list(qmd.db[qmd.db['TreeID']==tree].ModelID) #self.db[self.db['TreeID]==tree]['<Name>']
    treeTracker[tree]['Compared'] = []
    treeTracker[tree]['RankedList'] =[]
    for model in treeTracker[tree]['ModelIDs']:
        treeTracker[tree]['models'][model] = 0

In [ ]:
treeTracker[0]

In [ ]:
def launch_treeDB(db):
    treeDB = pd.DataFrame({
        'TreeID' : [],
        'ModelIDs' : [],
        'RankList' : [],
        'ModelScores' : [],
        'Compared' : []
    })
    
    for treeID in list(db['TreeID'].unique()):
        populateTreeDB(db, treeDB=treeDB, treeID=treeID)    
    return treeDB

def populateTreeDB(db, treeDB, treeID):
    
    new_row  = pd.Series({
        'TreeID' : treeID,
        'ModelIDs' : list(db[db['TreeID']==treeID].ModelID),
        'RankList' : [],
        'ModelScores' : [],
        'Compared' : [] # 2^model_a_id + 2^model_b_id ?
    })
    
    num_rows = len(treeDB)
    treeDB.loc[num_rows] = new_row

    
def addModelToTreeDB(db, treeDB, model_name):
    model_tree = DataBase.pull_field(db=db, name=model_name, field='TreeID')
    model_id = DataBase.pull_field(db=db, name=model_name, field='ModelID')
    tree_list_in_db = treeDB['TreeID']
    if model_tree not in tree_list_in_db:
        populateTreeDB(db, treeDB, model_tree)
    else: 
        if model_id in list(treeDB[treeDB['TreeID']==model_tree]['ModelIDs'])[0]:
            print("Model already in treeDB")
        else:
            list(treeDB[treeDB['TreeID']==treeID]['ModelIDs'])[0].append(model_id)    

In [ ]:
def update_field(db, field, name=None, model_id=None, new_value=None, increment=None):
    if name is not None: 
        db.loc[db['<Name>']==name, field] = new_value
    elif model_id is not None:
        db.loc[db['ModelID']==model_id, field] = new_value


In [ ]:
update_field(qmd.db, field='Status', name='z', new_value = 'Tets')

In [ ]:
qmd.db